In [1]:
%load_ext autoreload
%autoreload 2
# %cd /p/mmcardiac/Jerry/code/CINE-DENSE-Translation
%cd /home/jx8fh/Projects/LaMoD
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from modules.data import check_dict

/sfs/qumulo/qhome/jx8fh/Projects/LaMoD


In [5]:
# USE GPU 3
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. Load Data

In [3]:
test_fname = '/scratch/jx8fh/2024-06-22-MICCAI-ShapeMI-Workshop-data/lamod_test.npy'
test_data = np.load(test_fname, allow_pickle=True).tolist()

In [4]:
check_dict(test_data[0])

ori_n_frames                                                 51                  
augmented                                                    False               
DENSE_slice_mat_filename                                     /p/miauva/data/Jerry/medical-images/Cardiac-FromKen/Yu-DENSE-data/033_S01_base_auto.1_ROI.1.mat
DENSE_slice_location                                         76.947609505431     
subject_id                                                   033-DENSE           
slice_idx                                                    31                  
slice_full_id                                                033-31              
full_name                                                    033-DENSE_31        
DENSE_disp                                                   (2, 40, 48, 48)     
myo_masks                                                    (1, 40, 128, 128)   


# 2. Load Config

In [2]:
import json
config_fname = './configs/baseline-StrainNet.json'
config = json.load(open(config_fname))

# 3. Build Model

In [9]:
from models import build_model
checkpoint_fname = './checkpoints/baseline_StrainNet_motion_regression_he95gnwh.pth'
model = build_model(config['networks']['motion_regression'], skip_load_pretrained=True)
model.load_state_dict(torch.load(checkpoint_fname))
model = model.to(device)

Unrecognized kwargs: type


# 4. Inference

In [ ]:
from skimage.morphology import dilation, disk, closing
from skimage.transform import resize

# import torch.nn.functional as F
# from tqdm import tqdm
# from modules.data.datareader.DENSE_cine_IO_utils import align_n_frames_to
# from copy import deepcopy
# import torch


test_pred_data = []
config['training']['disp_masking'] = True
config['training']['resize_before_masking'] = False
mask_dilation = True
mask_closing = True
# mask_dilation = mask_closing = True
for datum in tqdm(cine_DENSE_data):
    if exps_infos[exp_name]['pixel_size'] == '1x1':
        myo_masks = datum['cine_cropped_myocardium_masks_rescaled_1mm2']
    else:
        myo_masks = datum['cine_cropped_myocardium_masks_rescaled_DENSE48']
    vol_ori_n_frames = myo_masks.shape[-1]
    

    DENSE_disp = np.stack([datum['DENSE_Lag_displacement_field_X'], datum['DENSE_Lag_displacement_field_Y']], axis=0)[...,:-1]

    DENSE_disp = align_n_frames_to(deepcopy(DENSE_disp), 40)
    DENSE_disp_tensor = torch.from_numpy(DENSE_disp).moveaxis(-1,1)[None]

    disp_scaling_factor = 1 if exps_infos[exp_name]['pixel_size'] == 'ori' else datum['DENSEInfo']['PixelSpacing'][0]
    
    disp_mask_src_48 = datum['cine_cropped_myocardium_masks_rescaled_DENSE48'][...,0]
    disp_mask_src_48_ori = deepcopy(datum['cine_cropped_myocardium_masks_rescaled_DENSE48'][...,0])
    if mask_dilation:
        disp_mask_src_48 = dilation(disp_mask_src_48, disk(2))
    if mask_closing:
        disp_mask_src_48 = closing(disp_mask_src_48, disk(2))
    disp_mask_src_48_vol = np.repeat(disp_mask_src_48[...,None], 40, axis=-1)
    disp_mask_src_48_tensor = torch.from_numpy(disp_mask_src_48_vol).moveaxis(-1, 0)[None, None].to(torch.float32)
    with torch.no_grad():
        if exp_name in ['MotionDecoder1x1']:
            vol_valid_n_frames = min(40, vol_ori_n_frames) - 1
            myo_masks = align_n_frames_to(deepcopy(myo_masks), 41)
            myo_masks_tensor = torch.from_numpy(myo_masks).moveaxis(-1,0)[None, None]    
            src_vol = myo_masks_tensor[:,:,0:1].repeat(1,1,40,1,1)
            tar_vol = myo_masks_tensor[:,:,1:]
            DENSE_disp[:,]
            batch = {
                'src': src_vol,
                'tar': tar_vol,
                'ori_n_frames': [vol_ori_n_frames],
                # 'DENSE_disp': DENSE_disp_pairs[patch_idx]
                'DENSE_disp': torch.cat([disp_mask_src_48_tensor, disp_mask_src_48_tensor], dim=1)
                # 'DENSE_disp': torch.cat([src, src], dim=1)
            }
        else:        
            vol_valid_n_frames = min(40, vol_ori_n_frames)
            myo_masks = align_n_frames_to(deepcopy(myo_masks), 40)
            myo_masks_tensor = torch.from_numpy(myo_masks).moveaxis(-1,0)[None, None]    
            batch = {
                'vol': myo_masks_tensor,
                'ori_n_frames': [vol_ori_n_frames],
                # 'DENSE_disp': DENSE_disp_pairs[patch_idx]
                'disp': torch.cat([disp_mask_src_48_tensor, disp_mask_src_48_tensor], dim=1)
                # 'DENSE_disp': torch.cat([src, src], dim=1)
            }
        total_loss, batch_loss_dict, batch_error_dict, pred_dict, target_dict = \
            trainer.batch_forward(
                batch, 
                networks, 
                'test', 
                mode='test', 
                train_config=config['training'], 
                full_config=config, 
                curr_epoch=-1)
        # DENSE_disp_GT = target_dict['DENSE_disp'].detach().cpu()
        DENSE_disp_pred = pred_dict['DENSE_disp'].detach().cpu() / disp_scaling_factor
        
        datum_pred = deepcopy(datum)
        datum_pred.update({
            'DENSE_disp_GT': DENSE_disp_tensor.cpu().numpy(),
            'DENSE_disp_pred': DENSE_disp_pred.cpu().numpy(),
            # 'reg_disp_pred': reg_disp_pred_merged,        
            'myo_masks_src_48': disp_mask_src_48,
            'myo_mask_src_48_ori': disp_mask_src_48_ori,
            'vol_ori_n_frames': vol_ori_n_frames,
            'vol_valid_n_frames': vol_valid_n_frames
        })
        # datum_pred = {
        #     'DENSE_disp_GT': DENSE_disp_tensor.cpu().numpy(),
        #     'DENSE_disp_pred': DENSE_disp_pred.cpu().numpy(),
        #     # 'reg_disp_pred': reg_disp_pred_merged,        
        #     'myo_masks_src_48': disp_mask_src_48,
        #     'vol_ori_n_frames': vol_ori_n_frames,
        #     'vol_valid_n_frames': vol_valid_n_frames
        # }            
        test_pred_data.append(datum_pred)
print(f'Saving {exp_name} prediction')
exps_infos[exp_name]['pred'] = test_pred_data
    # break